# Child Protection Data Visualisation, Sri Lanka

## Part 1: Processing Text-based PDFs

### 01.Loading PDF Files

Following Program Extract PDF table to CSV

In [4]:
import tabula

def pdf_to_csv(year):
    file_path = f'data/PDF-text/Statistical_data_-_Year_{year}.pdf'
    
    try:
        tables = tabula.read_pdf(file_path, pages='all', multiple_tables=True)
    except UnicodeDecodeError as e:
        print(f"Error reading PDF for year {year}: {e}. Skipping this file.")
        return  # Skip processing this file due to read error
    
    for index, table in enumerate(tables):
        csv_file_path = f'data/CSV/table_{index}_from_{year}.csv'
        excel_file_path = f'data/CSV/table_{index}_from_{year}.xlsx'

        try:
            table.to_csv(csv_file_path, index=False, encoding='utf-8', errors='replace')
        except UnicodeDecodeError as e:
            print(f"Error writing table {index} to CSV: {e}. Attempting to write to Excel instead.")
            try:
                table.to_excel(excel_file_path, index=False)
            except Exception as excel_error:
                print(f"Failed to write table {index} to Excel: {excel_error}")

for year in range(2017, 2021):
    pdf_to_csv(year)
    print(f'{year} processing completed.')

print('All files processing done.')


Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'
c:\Users\sdwee\anaconda3\lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


2017 processing completed.
2018 processing completed.
2019 processing completed.
2020 processing completed.
All files processing done.


above code gave an error for 'utf-8' encording related error for 2022 and 2023

In [ ]:
import pdfplumber
import pandas as pd

def pdf_to_csv_with_pdfplumber(year):
    file_path = f'data/PDF-text/Statistical_data_-_Year_{year}.pdf'
    with pdfplumber.open(file_path) as pdf:
        for i, page in enumerate(pdf.pages):
            # Attempt to extract tables from the page
            table = page.extract_table()
            if table:  # If a table is found
                df = pd.DataFrame(table[1:], columns=table[0])
                csv_file_path = f'data/CSV/table_{i}_from_{year}.csv'
                df.to_csv(csv_file_path, index=False)
                print(f"Table {i} from year {year} saved to CSV.")
            else:
                print(f"No table found on page {i+1} of year {year}.")

for year in range(2017, 2024):
    pdf_to_csv_with_pdfplumber(year)
    print(f'{year} processing completed with PDFPlumber.')

print('All files processing done with PDFPlumber.')

### 02.Text Data Preprocessing

#### The Reasons to Choose this pattern
Year, Month, District, and Number of Cases as Columns  
Structure: This would lead to a "long" table where each row represents a unique combination of year, month, and district, with another column for the number of cases. This is a more normalized format, often preferred for relational databases and time-series analyses.  
<b>Pros: </b>  
More flexible for various types of analysis, including time series, comparisons across districts, and aggregations (e.g., total cases per year, average cases per month).
Easier to manage with a large number of districts, as the table's width remains constant regardless of the number of districts.  
Cons:  
Can result in a very tall table, especially with data spanning several years, which might be slightly more cumbersome to navigate and visualize directly. However, this is often manageable with proper data analysis tools and techniques.

In [ ]:
import pandas as pd

i = 0
year = 2017

csv_file_path = f'data/CSV/table_{i}_from_{year}.csv'

df = pd.read_csv(csv_file_path)

# Add new Columns to df
df['Year'] = year
df['Month'] = None

# Column order
column_order =  ['Year', 'Month','District', 'No of complaints']

df = df[column_order]

df

In [3]:
def process_csv(i, year, main_df):
    # Construct the CSV file path
    csv_file_path = f'data/CSV/table_{i}_from_{year}.csv'

    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    # Add new columns to df
    df['Year'] = year
    if 'Month' not in df.columns:
        df['Month'] = None

    # Specify the column order
    column_order = ['Year', 'Month', 'District', 'No of complaints']

    # Reorder the columns
    df = df[column_order]

    # Concatenate the DataFrames
    main_df = pd.concat([main_df, df], ignore_index=True)


    return main_df

In [ ]:
df = pd.DataFrame()  # Initialize an empty DataFrame

for year in range(2017, 2022): 
    df = process_csv(0, year, df)
    print(f'{year} is complete')

print(df)

In [ ]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)

print(df)

### Preprocess data from 2022 pdf extraction csv

In [11]:
i =0
year = 2022
csv_file_path = f'data/CSV/table_from_{year}.csv'

df_1 = pd.read_csv(csv_file_path)

df_1

,Unnamed: 0,Unnamed: 1
0,TABLE 1,NaN
1,NaN,NaN
2,District,No of Cases
3,Ampara,234
4,Anuradhapura,597
5,Badulla,277
6,Batticaloa,160
7,Colombo,1708
8,Galle,703
9,Gampaha,1027


In [12]:
clean_df = df_1.dropna()

In [13]:
clean_df

,Unnamed: 0,Unnamed: 1
2,District,No of Cases
3,Ampara,234
4,Anuradhapura,597
5,Badulla,277
6,Batticaloa,160
7,Colombo,1708
8,Galle,703
9,Gampaha,1027
10,Hambanthota,395
11,Jaffna,166


In [16]:
# Assign the first row as the new header
clean_df.columns = clean_df.iloc[0]

#  Removing the previous header
clean_df = clean_df[1:]

In [17]:
clean_df

2,District,No of Cases
3,Ampara,234
4,Anuradhapura,597
5,Badulla,277
6,Batticaloa,160
7,Colombo,1708
8,Galle,703
9,Gampaha,1027
10,Hambanthota,395
11,Jaffna,166
12,Kaluthara,585


In [28]:
clean_df = clean_df.reset_index(drop=True)

clean_df

2,District,No of Cases
0,Ampara,234
1,Anuradhapura,597
2,Badulla,277
3,Batticaloa,160
4,Colombo,1708
5,Galle,703
6,Gampaha,1027
7,Hambanthota,395
8,Jaffna,166
9,Kaluthara,585


In [34]:
clean_df[clean_df['District'] == 'Total']

2,District,No of Cases
26,Total,10497
46,Total,10497


In [35]:
first_table_end_index = clean_df[clean_df['District'] == 'Total'].index[0]

In [31]:
table_01_df = clean_df.iloc[0:first_table_end_index+1]

table_01_df

2,District,No of Cases
0,Ampara,234
1,Anuradhapura,597
2,Badulla,277
3,Batticaloa,160
4,Colombo,1708
5,Galle,703
6,Gampaha,1027
7,Hambanthota,395
8,Jaffna,166
9,Kaluthara,585


In above dataframe we preprocess and make similar df format to the extract table 01 type from2017 to 2021.  
Below creating csv and run the main loop with process_csv() function.

In [42]:
# To keep the consistance of the all the csvs' change the column name 'No of Cases' to 'No of complaints'

table_01_df.rename(columns={'No of Cases': 'No of complaints'}, inplace=True)

C:\Users\sdwee\AppData\Local\Temp\ipykernel_13520\3992538943.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_01_df.rename(columns={'No of Cases': 'No of complaints'}, inplace=True)


In [43]:
table_01_df

2,District,No of complaints
0,Ampara,234
1,Anuradhapura,597
2,Badulla,277
3,Batticaloa,160
4,Colombo,1708
5,Galle,703
6,Gampaha,1027
7,Hambanthota,395
8,Jaffna,166
9,Kaluthara,585


In [44]:
i = 0
year =2022
csv_file_path = f'data/CSV/table_{i}_from_{year}.csv'
table_01_df.to_csv(csv_file_path, index=False)

Save table 02 for future uses

In [36]:
second_table_end_index = clean_df[clean_df['District'] == 'Total'].index[1]

table_02_df = clean_df.iloc[first_table_end_index+1: second_table_end_index+1]

table_02_df

2,District,No of Cases
27,Type of abuse,No of complaints
28,286A. Obscene publication,7
29,288. Procuring to beg,249
30,360B Sexual Exploitation of\nchildren,25
31,288B. Trafficking Restricted\nArticles,41
32,308A. Cruelty to Children,2096
33,345. Sexual Harassment,729
34,352. Kidnapping from lawful\nguardianship,131
35,353. Abduction,171
36,360C. Trafficking,132


In [39]:
# Assinging i raw as header

table_02_df.columns = table_02_df.iloc[0]

table_02_df = table_02_df[1:]

In [40]:
table_02_df

27,Type of abuse,No of complaints
28,286A. Obscene publication,7
29,288. Procuring to beg,249
30,360B Sexual Exploitation of\nchildren,25
31,288B. Trafficking Restricted\nArticles,41
32,308A. Cruelty to Children,2096
33,345. Sexual Harassment,729
34,352. Kidnapping from lawful\nguardianship,131
35,353. Abduction,171
36,360C. Trafficking,132
37,363. Rape,167


In [72]:
i = 1
year =2022
csv_file_path = f'data/CSV/table_{i}_from_{year}.pdf.csv'
table_02_df.to_csv(csv_file_path, index=False)

### Preprocess 2023 Data

In [51]:
i =3
year = 2023
csv_file_path = f'data/CSV/table_{i}_from_{year}.csv'

df = pd.read_csv(csv_file_path)

df

,No,District,January,February,March,April,May,June,July,August,September,October,November,December,Total
0,1,Ampara,20,17,22,11,21,22,16,13,18,29,23,25,237
1,2,Anuradapura,34,47,59,32,50,53,44,46,31,46,39,40,521
2,3,Badulla,16,21,28,16,23,18,22,18,24,19,23,18,246
3,4,Baticoloa,12,15,15,12,14,24,12,12,9,21,15,17,178
4,5,Colombo,112,100,143,75,111,112,97,78,86,98,87,75,1174
5,6,Galle,42,29,54,30,50,38,62,42,32,52,55,55,541
6,7,Gampaha,74,73,92,54,96,90,79,68,61,74,76,70,907
7,8,Hambanthota,24,30,40,16,36,34,43,31,32,30,40,37,18
8,9,Jaffna,10,13,6,7,18,14,13,5,10,14,18,10,138
9,10,Kaluthara,48,32,70,28,51,51,36,54,39,48,56,35,548


In [56]:

# Melt the DataFrame to convert 'January', 'February', etc. into a single 'Month' column
melted_df = pd.melt(df, id_vars=['No', 'District'], var_name='Month', value_name='No of complaints')

# drop No column
melted_df =  melted_df.drop('No', axis=1)

# Display the pivoted DataFrame
print(melted_df)

        District    Month  No of complaints
0         Ampara  January                20
1    Anuradapura  January                34
2        Badulla  January                16
3      Baticoloa  January                12
4        Colombo  January               112
..           ...      ...               ...
346   Rathnapura    Total               598
347    Trncomali    Total               133
348     Vavuniya    Total                65
349      Unknown    Total               434
350          NaN    Total              9436

[351 rows x 3 columns]


In [57]:
i = 0
year =2023
csv_file_path = f'data/CSV/table_{i}_from_{year}.csv'
melted_df.to_csv(csv_file_path, index=False)

### Now all the District wise complaint no tables are preprocessed. Lets combine all the data in the table into one dataframe and CSV.

From 2017 to 2023 table_0 have the same pattern, Beacause of that i am going to create one long table appending data from each csv.

In [58]:
def process_csv(i, year, main_df):
    # Construct the CSV file path
    csv_file_path = f'data/CSV/table_{i}_from_{year}.csv'

    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    # Add new columns to df
    df['Year'] = year
    if 'Month' not in df.columns:
        df['Month'] = None

    # Specify the column order
    column_order = ['Year', 'Month', 'District', 'No of complaints']

    # Reorder the columns
    df = df[column_order]

    # Concatenate the DataFrames
    main_df = pd.concat([main_df, df], ignore_index=True)


    return main_df

In [59]:
df = pd.DataFrame()  # Initialize an empty DataFrame

for year in range(2017, 2024): 
    df = process_csv(0, year, df)
    print(f'{year} is complete')

print(df)

2017 is complete
2018 is complete
2019 is complete
2020 is complete
2021 is complete
2022 is complete
2023 is complete
     Year  Month     District  No of complaints
0    2017   None       Ampara               208
1    2017   None  Anuradapura               441
2    2017   None      Badulla               198
3    2017   None    Baticoloa               179
4    2017   None      Colombo              1302
..    ...    ...          ...               ...
503  2023  Total   Rathnapura               598
504  2023  Total    Trncomali               133
505  2023  Total     Vavuniya                65
506  2023  Total      Unknown               434
507  2023  Total          NaN              9436

[508 rows x 4 columns]


In [60]:
i = 0
year =2023
csv_file_path = f'data/CSV/table_{i}_from_2017_to_2023.csv'
df.to_csv(csv_file_path, index=False)

## Preprocess table 02 that has complaints types and reported counts.

In [106]:
i = 1
year =2017
csv_file_path = f'data/CSV/table_{i}_from_{year}.pdf.csv' 
df = pd.read_csv(csv_file_path)
df

,Type of abuse,No of complaints
0,14. ACO - Restriction on payment,1
1,286A. Obscene publication,8
2,288. Procuring to beg,312
3,288B. Trafficking Restricted Articles,28
4,308A. Cruelty to Children,2144
5,345. Sexual Harassment,501
6,352. Kidnapping from lawful guardianship,145
7,353. Abduction,42
8,360C. Trafficking,116
9,360E. Soliciting a child,10


### We preprocess 2022 table 2 in previous section then, this section we have to preprocess 2023 cases type with month to give all years data a unform format

Child Abuse and Other child related complaints reported to NCPA by category - (Year 2023.01.01 to 2023.12.31) Df 0 and 1.

In [86]:
i = 0
year =2023
csv_file_path = f'data/CSV/table_{i}_from_{year}_.csv' 
df_0 = pd.read_csv(csv_file_path)
df_columns = df_0.columns
df_0

,No,Type of Abuse,January,February,March,April,May,June,July,August,September,October,November,December,Total
0,1,353 - Abduction,19,17,27,16,20,21,16,11,14,10,24,22,217
1,2,288 - Begging,21,32,24,17,36,38,28,21,25,30,27,24,323
2,3,CYPO 35 - in need of Care & Protection,159,169,294,140,246,213,225,204,185,224,208,259,2526
3,4,Child Labor,8,11,18,9,23,17,19,14,15,15,19,13,181
4,5,Compulsory Education,120,82,184,100,161,196,209,147,183,181,202,157,1922
5,6,308A - Cruelty,130,149,252,142,220,197,202,185,170,182,217,192,2238
6,7,365B - Grave Sexual Abuse,37,27,47,22,31,40,37,31,24,31,39,37,403
7,8,311 - Grievous Hurt,22,26,26,21,23,22,23,18,19,28,21,17,266
8,9,360 B - Sexual Exploitation,1,0,1,1,2,1,0,0,1,1,0,0,8
9,10,360D - Offences related to Adoption,1,0,0,0,0,0,2,0,0,0,0,0,3


In [87]:
df_columns

Index(['No', 'Type of Abuse', 'January', 'February', 'March', 'April', 'May',
       'June', 'July', 'August', 'September', 'October', 'November',
       'December', 'Total'],
      dtype='object')

In [88]:
i = 1
year =2023
csv_file_path = f'data/CSV/table_{i}_from_{year}_.csv' 
df_1 = pd.read_csv(csv_file_path, header=None)
df_1.columns = df_0.columns
df_1

,No,Type of Abuse,January,February,March,April,May,June,July,August,September,October,November,December,Total
0,29,"358A – Serfdom, Slavery, Child Soldiers",0,2,2,0,0,0,0,0,0,0,0,0,4
1,30,365 A - Gross Indecency,0,0,1,0,0,0,0,0,0,0,0,0,1
2,31,286 - C- Duty to inform of use of premises for...,0,0,1,0,0,0,0,0,0,0,0,0,1
3,32,360E – Soliciting a Child,0,0,0,0,0,0,0,0,0,1,0,0,1
4,33,365 C - Publication of Sexuel Abuse,0,0,0,0,4,0,0,0,0,0,0,0,4
5,34,CYPO 11 - Restrictions on reports of proseedin...,0,0,0,0,0,0,0,0,0,0,1,0,1
6,Total,NaN,663,620,1068,575,914,888,899,728,753,839,880,846,9673


In [89]:
# Concatenate the DataFrames
df = pd.concat([df_0, df_1], ignore_index=True)

df

,No,Type of Abuse,January,February,March,April,May,June,July,August,September,October,November,December,Total
0,1,353 - Abduction,19,17,27,16,20,21,16,11,14,10,24,22,217
1,2,288 - Begging,21,32,24,17,36,38,28,21,25,30,27,24,323
2,3,CYPO 35 - in need of Care & Protection,159,169,294,140,246,213,225,204,185,224,208,259,2526
3,4,Child Labor,8,11,18,9,23,17,19,14,15,15,19,13,181
4,5,Compulsory Education,120,82,184,100,161,196,209,147,183,181,202,157,1922
5,6,308A - Cruelty,130,149,252,142,220,197,202,185,170,182,217,192,2238
6,7,365B - Grave Sexual Abuse,37,27,47,22,31,40,37,31,24,31,39,37,403
7,8,311 - Grievous Hurt,22,26,26,21,23,22,23,18,19,28,21,17,266
8,9,360 B - Sexual Exploitation,1,0,1,1,2,1,0,0,1,1,0,0,8
9,10,360D - Offences related to Adoption,1,0,0,0,0,0,2,0,0,0,0,0,3


In [93]:
# Pivot above dataframe to 'Type of Abuse','Month', 'No of complaints'
melted_df = pd.melt(df, id_vars=['No', 'Type of Abuse'], var_name='Month', value_name='No of complaints')

# drop No column
melted_df =  melted_df.drop('No', axis=1)

# Display the pivoted DataFrame
print(melted_df)

                                         Type of Abuse    Month  \
0                                      353 - Abduction  January   
1                                        288 - Begging  January   
2               CYPO 35 - in need of Care & Protection  January   
3                                          Child Labor  January   
4                                 Compulsory Education  January   
..                                                 ...      ...   
450  286 - C- Duty to inform of use of premises for...    Total   
451                          360E – Soliciting a Child    Total   
452                365 C - Publication of Sexuel Abuse    Total   
453  CYPO 11 - Restrictions on reports of proseedin...    Total   
454                                                NaN    Total   

     No of complaints  
0                  19  
1                  21  
2                 159  
3                   8  
4                 120  
..                ...  
450                 1  
451

Write the 2023 file to a csv for future use.

In [94]:
# Write the 2023 file to a csv for future use.

i = 1
year =2023
csv_file_path = f'data/CSV/table_{i}_from_{year}.pdf.csv' 

melted_df.to_csv(csv_file_path)

Create Compalint type and count combine df and CSV.

In [2]:
def process_csv2(i, year, main_df):
    # Construct the CSV file path
    csv_file_path = f'data/CSV/table_{i}_from_{year}.pdf.csv'

    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    # Add new columns to df
    df['Year'] = year
    if 'Month' not in df.columns:
        df['Month'] = None

    # Specify the column order
    column_order = ['Year', 'Month', 'Type of abuse', 'No of complaints']

    # Reorder the columns
    df = df[column_order]

    # Concatenate the DataFrames
    main_df = pd.concat([main_df, df], ignore_index=True)


    return main_df

In [5]:
import pandas as pd

df = pd.DataFrame()  # Initialize an empty DataFrame

for year in range(2017, 2024): 
    df = process_csv2(1, year, df)
    print(f'{year} is complete')

print(df)

2017 is complete
2018 is complete
2019 is complete
2020 is complete
2021 is complete
2022 is complete
2023 is complete
     Year  Month                                      Type of abuse  \
0    2017   None                  14. ACO -  Restriction on payment   
1    2017   None                          286A. Obscene publication   
2    2017   None                              288. Procuring to beg   
3    2017   None              288B. Trafficking Restricted Articles   
4    2017   None                          308A. Cruelty to Children   
..    ...    ...                                                ...   
595  2023  Total  286 - C- Duty to inform of use of premises for...   
596  2023  Total                          360E – Soliciting a Child   
597  2023  Total                365 C - Publication of Sexuel Abuse   
598  2023  Total  CYPO 11 - Restrictions on reports of proseedin...   
599  2023  Total                                                NaN   

     No of complaints  
0   

In [6]:
i = 1
csv_file_path = f'data/CSV/table_{i}_from_2017_to_2023.csv'
df.to_csv(csv_file_path, index=False)

Child Abuse and Other child related offence wise (Miscellaneous Catogary) - (Year 2023.01.01 to 2023.12.31) df 2

In [95]:
i = 2
year =2023
csv_file_path = f'data/CSV/table_{i}_from_{year}_.csv' 
df_2 = pd.read_csv(csv_file_path)
df_2

,No,Type of Abuse,January,February,March,April,May,June,July,August,September,October,November,December,Total
0,1,Child Marriages,4,4,3,1,7,6,0,3,5,2,1,3,39
1,2,Cyber Bulling (threatening using nude photos),12,11,12,10,17,15,14,16,11,12,10,10,150
2,3,Domestic Violence,9,4,6,2,5,6,4,1,2,3,2,2,46
3,4,other (specify),63,76,65,60,65,57,65,54,46,48,49,43,691
4,5,Abortion,1,0,0,1,0,0,0,1,0,0,0,0,3
5,6,Attempt to commit suicide,2,0,2,1,1,1,1,3,1,3,2,8,25
6,7,Need Counseling/psycho-social support,17,27,41,19,41,35,36,32,37,45,63,61,454
7,8,Access for children,1,0,1,0,0,0,0,0,0,0,0,0,2
8,9,Drug Abuse,9,7,5,8,12,11,14,14,6,10,16,13,125
9,10,Inability to send to school due to financial o...,1,1,2,1,0,3,0,0,1,1,0,0,10


In [96]:
# Pivot above dataframe to 'Type of Abuse','Month', 'No of complaints'
melted_df = pd.melt(df_2, id_vars=['No', 'Type of Abuse'], var_name='Month', value_name='No of complaints')

# drop No column
melted_df =  melted_df.drop('No', axis=1)

# Display the pivoted DataFrame
print(melted_df)

                                     Type of Abuse    Month  No of complaints
0                                  Child Marriages  January                 4
1    Cyber Bulling (threatening using nude photos)  January                12
2                                Domestic Violence  January                 9
3                                  other (specify)  January                63
4                                         Abortion  January                 1
..                                             ...      ...               ...
255                               Violent Behavior    Total                27
256                          No Birth Certificates    Total                37
257                         Refuse to go to school    Total                 6
258                    Police Assult and detention    Total                 3
259                                            NaN    Total              1714

[260 rows x 3 columns]


In [97]:
# Write Child Abuse and Other child related offence wise (Miscellaneous Catogary) - (Year 2023.01.01 to 2023.12.31) to a csv

i = 2
year =2023
csv_file_path = f'data/CSV/table_{i}_.pdf.csv' 

melted_df.to_csv(csv_file_path, index=False)

Update Sri lankan District name with standed names, and replace wrong spelled names.

In [ ]:
# Read CSV with district

df = pd.read_csv('data/CSV/table_0_from_2017_to_2023.csv')

df

In [126]:
type(df['District'].unique())

arr = df['District'].unique()
arr

array(['Ampara', 'Anuradapura', 'Badulla', 'Baticoloa', 'Colombo',
       'Galle', 'Gampaha', 'Hambanthota', 'Jaffna', 'Kaluthara', 'Kandy',
       'Kegalle', 'Kilinochchi', 'Kurunegala', 'Mannar', 'Matale',
       'Matara', 'Monaragala', 'Mulathivu', 'Nuwara Eliya', 'Polonnaruwa',
       'Puttlum', 'Rathnapura', 'Trincomali', 'Vavuniya', 'Total',
       'NuwaraEliya', 'Anuradhapura', 'Batticaloa', 'Kagalle',
       'Kurunagala', 'Mullaithivu', 'Puttalama', 'Trincomalee',
       'All Island', 'Trncomali', 'Unknown', nan], dtype=object)

In [127]:
len(arr)

38

In [129]:
# Dictionary mapping incorrect names to their correct forms
correction_map = {
    'Anuradapura': 'Anuradhapura',
    'Baticoloa': 'Batticaloa',
    'Kaluthara': 'Kalutara',
    'Kegalle': 'Kegalle',  # This seems correct, included for completeness
    'Kilinochchi': 'Kilinochchi',  # This seems correct, included for completeness
    'Kurunegala': 'Kurunegala',  # This seems correct, included for completeness
    'Mulathivu': 'Mullaitivu',
    'Nuwara Eliya': 'Nuwara Eliya',  # This seems correct, included for completeness
    'Puttlum': 'Puttalam',
    'Rathnapura': 'Ratnapura',
    'Trincomali': 'Trincomalee',
    'Vavuniya': 'Vavuniya',  # This seems correct, included for completeness
    'NuwaraEliya': 'Nuwara Eliya',
    'Kagalle': 'Kegalle',
    'Kurunagala': 'Kurunegala',
    'Mullaithivu': 'Mullaitivu',
    'Puttalama': 'Puttalam',
    'Trncomali': 'Trincomalee',
}

# Apply correction to the 'district' column
df['District'] = df['District'].map(correction_map).fillna(df['District'])

df

,Year,Month,District,No of complaints
0,2017,NaN,Ampara,208
1,2017,NaN,Anuradhapura,441
2,2017,NaN,Badulla,198
3,2017,NaN,Batticaloa,179
4,2017,NaN,Colombo,1302
...,...,...,...,...
503,2023,Total,Ratnapura,598
504,2023,Total,Trincomalee,133
505,2023,Total,Vavuniya,65
506,2023,Total,Unknown,434


In [130]:
df['District'].unique()

array(['Ampara', 'Anuradhapura', 'Badulla', 'Batticaloa', 'Colombo',
       'Galle', 'Gampaha', 'Hambanthota', 'Jaffna', 'Kalutara', 'Kandy',
       'Kegalle', 'Kilinochchi', 'Kurunegala', 'Mannar', 'Matale',
       'Matara', 'Monaragala', 'Mullaitivu', 'Nuwara Eliya',
       'Polonnaruwa', 'Puttalam', 'Ratnapura', 'Trincomalee', 'Vavuniya',
       'Total', 'All Island', 'Unknown', nan], dtype=object)

In [131]:
len(df['District'].unique())

29

Replace the duplicated name with standard , spelling for sri lankan district names and update following csv

In [133]:
df.to_csv('data/CSV/table_0_from_2017_to_2023.csv')

### Preprocess duplicate names for type of cases with latest or standed names

In [7]:
# Read CSV with Cases types

df = pd.read_csv('data/CSV/table_1_from_2017_to_2023.csv')

df

,Year,Month,Type of abuse,No of complaints
0,2017,NaN,14. ACO - Restriction on payment,1.0
1,2017,NaN,286A. Obscene publication,8.0
2,2017,NaN,288. Procuring to beg,312.0
3,2017,NaN,288B. Trafficking Restricted Articles,28.0
4,2017,NaN,308A. Cruelty to Children,2144.0
...,...,...,...,...
595,2023,Total,286 - C- Duty to inform of use of premises for...,1.0
596,2023,Total,360E – Soliciting a Child,1.0
597,2023,Total,365 C - Publication of Sexuel Abuse,4.0
598,2023,Total,CYPO 11 - Restrictions on reports of proseedin...,1.0


In [8]:
df['Type of abuse'].unique()

array(['14. ACO -  Restriction on payment', '286A. Obscene publication',
       '288. Procuring to beg', '288B. Trafficking Restricted Articles',
       '308A. Cruelty to Children', '345. Sexual Harassment',
       '352. Kidnapping from lawful guardianship', '353. Abduction',
       '360C. Trafficking', '360E. Soliciting a child', '363. Rape',
       '364A. Incest', '365. Unnatural Offence', '365A. Gross Indecency',
       '365B. Grave sexual abuse', '71. CYPO -  Neglect of Children',
       '76. CYPO -  Sale of tobacco', 'Child Labor',
       'Compulsory Education', 'Domestic Violence',
       'Juvenile Delinquency', 'MISC.', 'Total',
       '27A. ACO - Unlawful Custody', '308. Exposure and Abandonment',
       '360B. Sexual Exploitation of Children',
       '27A. ACO - Unlawful custody',
       '35. CYPO -  Child or Young Person in need of C&P',
       'Miscellaneous', 'Cruelty to Children', 'Sexual Harassment',
       'Children and Young Person Ordinance-Neglect of Children',
      

In [151]:
sorted_list = sorted(map(str,df['Type of abuse'].unique()))

sorted_list

['14. ACO -  Restriction on payment',
 '27A. ACO - Unlawful Custody',
 '27A. ACO - Unlawful custody',
 '286 - A- Obscene Publications',
 '286 - C- Duty to inform of use of premises for\r\nchild abuse',
 '286A. Obscene publication',
 '288 - Begging',
 '288. Procuring to beg',
 '288A - Hiring or employing children to act as procures\r\nfor sexual intercourse.',
 '288B - Restricted Articles/Drugs',
 '288B. Trafficking Restricted\nArticles',
 '288B. Trafficking Restricted Articles',
 '308. Exposure and Abandonment',
 '308A - Cruelty',
 '308A. Cruelty to Children',
 '310 - Cause hurt',
 '311 - Grievous Hurt',
 '345 - Sexual Harassment',
 '345. Sexual Harassment',
 '35. CYPO -  Child or Young Person in need of C&P',
 '351 - Kidnapping from Sri Lanka',
 '352 - Kidnapping from lawful guardianship',
 '352. Kidnapping from lawful\nguardianship',
 '352. Kidnapping from lawful guardianship',
 '353 - Abduction',
 '353. Abduction',
 '356 - Kidnapping or abducting with intent secretly\r\nand wrongful

In [152]:
len(sorted_list)

84

In [154]:
sorted_list = sorted(map(str,df['Type of abuse'].str.upper().unique()))

sorted_list

['14. ACO -  RESTRICTION ON PAYMENT',
 '27A. ACO - UNLAWFUL CUSTODY',
 '286 - A- OBSCENE PUBLICATIONS',
 '286 - C- DUTY TO INFORM OF USE OF PREMISES FOR\r\nCHILD ABUSE',
 '286A. OBSCENE PUBLICATION',
 '288 - BEGGING',
 '288. PROCURING TO BEG',
 '288A - HIRING OR EMPLOYING CHILDREN TO ACT AS PROCURES\r\nFOR SEXUAL INTERCOURSE.',
 '288B - RESTRICTED ARTICLES/DRUGS',
 '288B. TRAFFICKING RESTRICTED\nARTICLES',
 '288B. TRAFFICKING RESTRICTED ARTICLES',
 '308. EXPOSURE AND ABANDONMENT',
 '308A - CRUELTY',
 '308A. CRUELTY TO CHILDREN',
 '310 - CAUSE HURT',
 '311 - GRIEVOUS HURT',
 '345 - SEXUAL HARASSMENT',
 '345. SEXUAL HARASSMENT',
 '35. CYPO -  CHILD OR YOUNG PERSON IN NEED OF C&P',
 '351 - KIDNAPPING FROM SRI LANKA',
 '352 - KIDNAPPING FROM LAWFUL GUARDIANSHIP',
 '352. KIDNAPPING FROM LAWFUL\nGUARDIANSHIP',
 '352. KIDNAPPING FROM LAWFUL GUARDIANSHIP',
 '353 - ABDUCTION',
 '353. ABDUCTION',
 '356 - KIDNAPPING OR ABDUCTING WITH INTENT SECRETLY\r\nAND WRONGFULLY TO CONFINE A PERSON',
 '358A 

In [155]:
len(sorted_list)

83

In [ ]:
# Following methods did not work properly
import pandas as pd

# Read CSV with Cases types
df = pd.read_csv('data/CSV/table_1_from_2017_to_2023.csv')

# Extract 'Type of abuse' column
type_of_abuse_column = df['Type of abuse']

# Convert 'Type of abuse' column to uppercase
#type_of_abuse_column = type_of_abuse_column.str.upper()

# Replace hyphens with dots
#type_of_abuse_column = type_of_abuse_column.str.replace('-', '.')

# Remove leading/trailing whitespaces and replace consecutive whitespaces with a single space
#type_of_abuse_column = type_of_abuse_column.str.replace('\s+', ' ').str.strip()

# Remove spaces around dots
type_of_abuse_column = type_of_abuse_column.str.replace('\.\s+', '.')

# Drop duplicate values, keeping the last occurrence
type_of_abuse_column.drop_duplicates(keep='last', inplace=True)

# Replace the original 'Type of abuse' column with the cleaned column
df['Type of abuse'] = type_of_abuse_column

# Print the updated DataFrame

df['Type of abuse'] 


In [35]:
correction_map = {
    '14. ACO -  Restriction on payment': '14 - ACO: Restriction on Payment',
    '286A. Obscene publication': '286A - Obscene Publication',
    '288. Procuring to beg': '288 - Procuring to Beg',
    '288B. Trafficking Restricted Articles': '288B - Trafficking Restricted Articles',
    '308A. Cruelty to Children': '308A - Cruelty to Children',
    '345. Sexual Harassment': '345 - Sexual Harassment',
    '352. Kidnapping from lawful guardianship': '352 - Kidnapping from Lawful Guardianship',
    '353. Abduction': '353 - Abduction',
    '360C. Trafficking': '360C - Trafficking',
    '360E. Soliciting a child': '360E - Soliciting a Child',
    '363. Rape': '363 - Rape',
    'Rape': '363 - Rape',
    '364A. Incest': '364A - Incest',
    '365. Unnatural Offence': '365 - Unnatural Offence',
    '365A. Gross Indecency': '365A - Gross Indecency',
    '365B. Grave sexual abuse': '365B - Grave Sexual Abuse',
    '76 - CYPO: Sale of Tobacco':'76 - CYPO: Sale of Tobacco to Person Under 16 Years',
    '76. CYPO -  Sale of tobacco':'76 - CYPO: Sale of Tobacco to Person Under 16 Years',
    'CYPO 76 - sale of tobacco to person under 16 years': '76 - CYPO: Sale of Tobacco to Person Under 16 Years',
    '76 - CYPO :sale of tobacco to person under 16 years': '76 - CYPO: Sale of Tobacco to Person Under 16 Years',
    'Child Labor': 'Child Labor',
    'Compulsory Education': 'Compulsory Education',
    'Domestic Violence': 'Domestic Violence',
    'Juvenile Delinquency': 'Juvenile Delinquency',
    'MISC.': 'Miscellaneous',
    'Total': 'Total',
    '27A. ACO - Unlawful Custody': '27A - ACO: Unlawful Custody',
    '308. Exposure and Abandonment': '308 - Exposure and Abandonment',
    '360B. Sexual Exploitation of Children': '360B - Sexual Exploitation of Children',
    '27A. ACO - Unlawful custody': '27A - ACO: Unlawful Custody',
    '35. CYPO -  Child or Young Person in need of C&P': '35 - CYPO: Child or Young Person in Need of Care & Protection',
    'Miscellaneous': 'Miscellaneous',
    'Children and Young Person Ordinance ?': 'Children and Young Person Ordinance',
    'Neglect of Children': 'Neglect of Children',
    'Grave sexual abuse': '365B - Grave Sexual Abuse',
    'Child Labour': 'Child Labor',
    'Procuring to beg': '288 - Procuring to Beg',
    'Kidnapping from lawful guardianship': '352 - Kidnapping from Lawful Guardianship',
    'Trafficking': '360C - Trafficking',
    'Abduction': '353 - Abduction',
    'Trafficking Restricted Articles': '288B - Trafficking Restricted Articles',
    'Obscene publication': '286A - Obscene Publication',
    'Soliciting a child': '360E - Soliciting a Child',
    'Incest': '364A - Incest',
    'Unnatural Offence': '365 - Unnatural Offence',
    '360B Sexual Exploitation of children': '360B - Sexual Exploitation of Children',
    '360B Sexual Exploitation of\nchildren': '360B - Sexual Exploitation of Children',
    '288B. Trafficking Restricted\nArticles': '288B - Trafficking Restricted Articles',
    '352. Kidnapping from lawful\nguardianship': '352 - Kidnapping from Lawful Guardianship',
    '71. CYPO - Neglect of\nChildren': '71 - CYPO: Neglect of Children, Cruelty',
    '71 - CYPO: Neglect of Children':'71 - CYPO: Neglect of Children, Cruelty',
    '71. CYPO -  Neglect of Children':  '71 - CYPO: Neglect of Children, Cruelty',
    'CYPO 71 – Neglect of children,Crualty': '71 - CYPO: Neglect of Children, Cruelty',
    'MISC': 'Miscellaneous',
    '353 - Abduction': '353 - Abduction',
    '288 - Begging': '288 - Begging',
    'CYPO 35 - in need of Care & Protection': '35 - CYPO: Child or Young Person in Need of Care & Protection',
    '308A - Cruelty': '308A - Cruelty to Children',
    'Cruelty to Children': '308A - Cruelty to Children',
    '365B - Grave Sexual Abuse': '365B - Grave Sexual Abuse',
    '311 - Grievous Hurt': '311 - Grievous Hurt',
    '360 B - Sexual Exploitation': '360B - Sexual Exploitation of Children',
    '360D - Offences related to Adoption': '360D - Offences Related to Adoption',
    '364A - Incest': '364A - Incest',
    '363 - Rape': '363 - Rape',
    'Juvenile Delinqency': 'Juvenile Delinquency',
    '352 - Kidnapping from lawful guardianship': '352 - Kidnapping from Lawful Guardianship',
    '360C - Trafficking': '360C - Trafficking',
    '360A - Procreation': '360A - Procreation',
    '288B - Restricted Articles/Drugs': '288B - Trafficking Restricted Articles/Drugs',
    '345 - Sexual Harassment': '345 - Sexual Harassment',
    'Sexual Harassment': '345 - Sexual Harassment',
    '288A - Hiring or employing children to act as procures\r\nfor sexual intercourse.': '288A - Hiring or Employing Children to Act as Procurers for Sexual Intercourse',
    '310 - Cause hurt': '310 - Cause Hurt',
    '351 - Kidnapping from Sri Lanka': '351 - Kidnapping from Sri Lanka',
    '356 - Kidnapping or abducting with intent secretly\r\nand wrongfully to confine a person': '356 - Kidnapping or Abducting with Intent Secretly and Wrongfully to Confine a Person',
    'CYPO 20 – Prohibition of publication of certain\r\nmatters': 'CYPO 20 - Prohibition of Publication of Certain Matters',
    'CYPO 72- seduction or prostitution of girl under 16y': 'CYPO 72 - Seduction or Prostitution of Girl Under 16 Years',
    'CYPO 77 - preventing children from receiving\r\neducation': 'CYPO 77 - Preventing Children from Receiving Education',
    '286 - A- Obscene Publications': '286A - Obscene Publications',
    '358A – Serfdom, Slavery, Child Soldiers': '358A - Serfdom, Slavery, Child Soldiers',
    '365 A - Gross Indecency': '365A - Gross Indecency',
    '286 - C- Duty to inform of use of premises for\r\nchild abuse': '286C - Duty to Inform of Use of Premises for Child Abuse',
    '360E – Soliciting a Child': '360E - Soliciting a Child',
    '365 C - Publication of Sexuel Abuse': '365C - Publication of Sexual Abuse',
    'CYPO 11 - Restrictions on reports of proseedings in\r\njuvenile courts': 'CYPO 11 - Restrictions on Reports of Proceedings in Juvenile Courts',
    '286A - Obscene Publication':'286A - Obscene Publications',
    '288 - Begging':'288 - Procuring to Beg',
    '288B - Trafficking Restricted Articles':'288B - Trafficking Restricted Articles/Drugs',
    
}


# Apply correction to the 'Type of abuse' column
df['Type of abuse'] = df['Type of abuse'].replace(correction_map)

# Print the updated DataFrame
print(df)


     Year  Month                                      Type of abuse  \
0    2017    NaN                   14 - ACO: Restriction on Payment   
1    2017    NaN                        286A - Obscene Publications   
2    2017    NaN                             288 - Procuring to Beg   
3    2017    NaN       288B - Trafficking Restricted Articles/Drugs   
4    2017    NaN                         308A - Cruelty to Children   
..    ...    ...                                                ...   
595  2023  Total  286C - Duty to Inform of Use of Premises for C...   
596  2023  Total                          360E - Soliciting a Child   
597  2023  Total                 365C - Publication of Sexual Abuse   
598  2023  Total  CYPO 11 - Restrictions on Reports of Proceedin...   
599  2023  Total                                                NaN   

     No of complaints  
0                 1.0  
1                 8.0  
2               312.0  
3                28.0  
4              2144.0  
.. 

In [20]:
sorted_list = sorted(map(str,df['Type of abuse'].unique()))

sorted_list

['14 - ACO: Restriction on Payment',
 '27A - ACO: Unlawful Custody',
 '286A - Obscene Publications',
 '286C - Duty to Inform of Use of Premises for Child Abuse',
 '288 - Procuring to Beg',
 '288A - Hiring or Employing Children to Act as Procurers for Sexual Intercourse',
 '288B - Trafficking Restricted Articles/Drugs',
 '308 - Exposure and Abandonment',
 '308A - Cruelty to Children',
 '310 - Cause Hurt',
 '311 - Grievous Hurt',
 '345 - Sexual Harassment',
 '35 - CYPO: Child or Young Person in Need of Care & Protection',
 '351 - Kidnapping from Sri Lanka',
 '352 - Kidnapping from Lawful Guardianship',
 '353 - Abduction',
 '356 - Kidnapping or Abducting with Intent Secretly and Wrongfully to Confine a Person',
 '358A - Serfdom, Slavery, Child Soldiers',
 '360A - Procreation',
 '360B - Sexual Exploitation of Children',
 '360C - Trafficking',
 '360D - Offences Related to Adoption',
 '360E - Soliciting a Child',
 '363 - Rape',
 '364A - Incest',
 '365 - Unnatural Offence',
 '365A - Gross Ind

In [21]:
len(sorted_list)

46

In [22]:
df

,Year,Month,Type of abuse,No of complaints
0,2017,NaN,14 - ACO: Restriction on Payment,1.0
1,2017,NaN,286A - Obscene Publications,8.0
2,2017,NaN,288 - Procuring to Beg,312.0
3,2017,NaN,288B - Trafficking Restricted Articles/Drugs,28.0
4,2017,NaN,308A - Cruelty to Children,2144.0
...,...,...,...,...
595,2023,Total,286C - Duty to Inform of Use of Premises for C...,1.0
596,2023,Total,360E - Soliciting a Child,1.0
597,2023,Total,365C - Publication of Sexual Abuse,4.0
598,2023,Total,CYPO 11 - Restrictions on Reports of Proceedin...,1.0


In [36]:
df.to_csv('data/CSV/table_1_from_2017_to_2023.csv',  index=False)

Catagorize abuse type according to "International Classification of Violence against Children"  PDF.  
https://data.unicef.org/wp-content/uploads/2023/07/International-Classification-of-Violence-against-Children.pdf

In [37]:
import pandas as pd

# Define the abuse types and their corresponding categories based on the provided information
data = {
    "Type of Abuse": [
        "Sexual Harassment", "Miscellaneous", "Juvenile Delinquency", "Domestic Violence",
        "CYPO 77 - Preventing Children from Receiving Education", 
        "CYPO 72 - Seduction or Prostitution of Girl Under 16 Years",
        "CYPO 20 - Prohibition of Publication of Certain Matters",
        "CYPO 11 - Restrictions on Reports of Proceedings in Juvenile Courts",
        "Cruelty to Children", "Compulsory Education", 
        "Children and Young Person Ordinance-Neglect of Children", "Child Labor",
        "365C - Publication of Sexual Abuse", "365B - Grave Sexual Abuse",
        "365A - Gross Indecency", "365 - Unnatural Offence", "364A - Incest",
        "363 - Rape", "360E - Soliciting a Child", "360D - Offences Related to Adoption",
        "360C - Trafficking", "360B - Sexual Exploitation of Children", "360A - Procreation",
        "358A - Serfdom, Slavery, Child Soldiers",
        "356 - Kidnapping or Abducting with Intent Secretly and Wrongfully to Confine a Person",
        "353 - Abduction", "352 - Kidnapping from Lawful Guardianship", 
        "351 - Kidnapping from Sri Lanka", "345 - Sexual Harassment", 
        "311 - Grievous Hurt", "310 - Cause Hurt", "308A - Cruelty to Children",
        "308 - Exposure and Abandonment", "288B - Trafficking Restricted Articles/Drugs",
        "288A - Hiring or Employing Children to Act as Procurers for Sexual Intercourse",
        "288 - Procuring to Beg", "286C - Duty to Inform of Use of Premises for Child Abuse",
        "286A - Obscene Publications", "76 - CYPO: Sale of Tobacco to Person Under 16 Years",
        "71 - CYPO: Neglect of Children, Cruelty", 
        "35 - CYPO: Child or Young Person in Need of Care & Protection", 
        "27A - ACO: Unlawful Custody", "14 - ACO: Restriction on Payment"
    ],
    "Category": [
        "Sexual Violence Against a Child", "Miscellaneous", "Miscellaneous", "Psychological Violence Against a Child",
        "Neglect of a Child", "Sexual Violence Against a Child", "Miscellaneous", "Miscellaneous",
        "Physical Violence Against a Child", "Neglect of a Child", "Neglect of a Child", "Child Labor and Economic Exploitation",
        "Sexual Violence Against a Child", "Sexual Violence Against a Child", "Sexual Violence Against a Child", 
        "Sexual Violence Against a Child", "Sexual Violence Against a Child", "Sexual Violence Against a Child", 
        "Sexual Violence Against a Child", "Sexual Violence Against a Child", "Sexual Violence Against a Child", 
        "Sexual Violence Against a Child", "Sexual Violence Against a Child", "Other Specific Acts That Could Result in Harm to the Child",
        "Other Specific Acts That Could Result in Harm to the Child", "Other Specific Acts That Could Result in Harm to the Child", 
        "Other Specific Acts That Could Result in Harm to the Child", "Other Specific Acts That Could Result in Harm to the Child", 
        "Sexual Violence Against a Child", "Physical Violence Against a Child", "Physical Violence Against a Child", 
        "Physical Violence Against a Child", "Physical Violence Against a Child", 
        "Other Specific Acts That Could Result in Harm to the Child", "Other Specific Acts That Could Result in Harm to the Child", 
        "Other Specific Acts That Could Result in Harm to the Child", "Other Specific Acts That Could Result in Harm to the Child", 
        "Other Specific Acts That Could Result in Harm to the Child", "Other Specific Acts That Could Result in Harm to the Child", 
        "Neglect of a Child", "Neglect of a Child", "Psychological Violence Against a Child", "Psychological Violence Against a Child"
    ]
}

# Create a DataFrame
#df = pd.DataFrame(abuse_data)

# Save the DataFrame to a CSV file
csv_file_path = "data/CSV/child_abuse_classification.csv"
df.to_csv(csv_file_path, index=False)
